<a href="https://colab.research.google.com/github/nguyen-nhat-mai/sentiment_analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Environment set up

In [ ]:
!pip install torch==1.13.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 61.8 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1+cu118 requires torch==2.0.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.1 requires torch==2.0.0, but you have torch

In [ ]:
import torch
print(torch.__version__)

1.13.1+cu117


In [ ]:
!pip install pytorch-lightning==1.8.1
!pip install transformers==4.22.2
!pip install datasets==2.9.0
!pip install sentencepiece==0.1.97
!pip install scikit-learn==1.2.0
!pip install numpy==1.23.5
!pip install pandas==1.5.3
!pip install nltk==3.8.1
!pip install stanza==1.4.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.4/798.4 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 21.5 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116952 sha256=80c8fddbb172a2dc738ca27cd83d255fec2933b47050f01e6af5ad471d360e2a
  Stored in directory: /root/.cache/pip/wheels/f7/f1/89/b9ea2bf8f80ec027a88fef1d354b3816b4d3d29530988972f6
Successfully built fire
Looking in ind

In [ ]:
import transformers
from transformers import BertModel, BertTokenizer
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
from collections import defaultdict
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
import torch.nn.functional as F
import random as rn
from random import shuffle
import time, sys
np.random.seed(17)
rn.seed(12345)

## Test

In [ ]:
# Network
class SentimentCheck(nn.Module):
    
    def __init__(self, n_classes=3):
        super(SentimentCheck, self).__init__()
        self.model = BertModel.from_pretrained('bert-base-uncased')
        self.drop = nn.Dropout(p=0.45)
        self.fc = nn.Linear(self.model.config.hidden_size, n_classes)
        
    def forward(self, input_ids, attention_mask):
        _, output = self.model(input_ids = input_ids, attention_mask = attention_mask)[0:2]
        output = self.drop(output)
        output = self.fc(output)
        return output

In [ ]:
# Preprocessing

#----------------PREPROCESS RAW DATA----------------#
# Converting aspects to questions and consider sentences as replies (account for context)

aspect_map = {"AMBIENCE#GENERAL":"What do you think of the ambience ?",
              "FOOD#QUALITY":"What do you think of the quality of the food ?",
              "SERVICE#GENERAL":"What do you think of the service ?",
              "FOOD#STYLE_OPTIONS": "What do you think of the food choices ?",
              "DRINKS#QUALITY":"What do you think of the drinks?",
              'RESTAURANT#MISCELLANEOUS': "What do you think of the restaurant ?",
              'RESTAURANT#GENERAL': "What do you think of the restaurant ?",
              'LOCATION#GENERAL': 'What do you think of the location ?',
              'DRINKS#STYLE_OPTIONS': "What do you think of the drink choices ?",
              'RESTAURANT#PRICES':'What do you think of the price of it ?',
              'DRINKS#PRICES':'What do you think of the price of it ?',
              'FOOD#PRICES': 'What do you think of the price of it ?'            
              }
label_encode = {"negative":0,"neutral":1,"positive":2}

def preprocessing(file):
    file[0] = file[0].apply(lambda x: label_encode[x])
    file[1] = file[1].apply(lambda x: aspect_map[x])
    return file

#----------------PREPARE PYTORCH DATASET----------------#
# Function to calculate max token length as input for creating pytorch dataset
def max_len_token(file, tokenizer):
    token_lengths = []
    for sentence in file[4]:
        tokens = tokenizer.encode(sentence, max_length=1000)
        token_lengths.append(len(tokens))
    return max(token_lengths)

# Create pytorch dataset
class create_dataset(Dataset):
    
    def __init__(self, sentences, aspect_categories, target_terms, labels, tokenizer, max_len_token):
        
        self.sentences = sentences
        self.aspect_categories = aspect_categories
        self.target_terms = target_terms
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len_token = max_len_token
        
        
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, index):
        sentence = str(self.sentences[index])
        aspect_category = str(self.aspect_categories[index])
        target_term = str(self.target_terms[index])
        label = self.labels[index]

        aspect = aspect_category + ' ' + target_term
        
        # Get the attention maks
        encoding = self.tokenizer.encode_plus(
            text = sentence,
            text_pair = aspect,
            add_special_tokens=True, # Add '[CLS]' and '[SEP]'
            max_length=self.max_len_token, # Pad & truncate all sentences.
            return_token_type_ids=False,
            pad_to_max_length=True, # Pad to the end of the sentence
            return_attention_mask=True, # Construct attn. masks.
            return_tensors='pt',  # Return PyTorch tensors
            truncation=True)

        return {
            'sentence_text': sentence,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)}

In [ ]:
# Model
class Classifier:
    """
    The Classifier: complete the definition of this class template by providing a constructor (i.e. the
    __init__() function) and the 2 methods train() and predict() below. Please donot change
     """

    ###############################################################################
    def train(self, train_filename: str, dev_filename: str, device: torch.device):
        """
        Trains the classifier model on the training set stored in file trainfile
        PLEASE:
          - DO NOT CHANGE THE SIGNATURE OF THIS METHOD
          - PUT THE MODEL and DATA on the specified device! Do not use another device
          - DO NOT USE THE DEV DATA AS TRAINING EXAMPLES, YOU CAN USE THEM ONLY FOR THE OPTIMIZATION
         OF MODEL HYPERPARAMETERS
        """
        
        #----------------LOAD FILE & PREPROCESS----------------#
        # Load the files into pandas dataframe
        train_file = pd.read_csv(train_filename, sep='\t', header=None)
        if dev_filename is not None:
            dev_file = pd.read_csv(dev_filename, sep='\t', header=None)
        
        # Proprocess the file
        self.trainfile = preprocessing(train_file)
        if dev_filename is not None:
            self.dev_file = preprocessing(dev_file)
        
        #----------------ENCODING & GET DATALOADER------------------#
        # Set the pretrained model & tokenizing
        PRE_TRAINED_MODEL = 'bert-base-uncased'
        self.tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL)
        
        # Load the dataloader
        self.bs = 16 # Set batch size
        self.max_length = max_len_token(self.trainfile, self.tokenizer)
        self.train_dataset = create_dataset(
            sentences=self.trainfile[4].to_numpy(),
            aspect_categories = self.trainfile[1].to_numpy(),
            target_terms=self.trainfile[2].to_numpy(),
            labels = self.trainfile[0].to_numpy(),
            tokenizer = self.tokenizer,
            max_len_token = self.max_length)
        self.train_loader = DataLoader(self.train_dataset, batch_size= self.bs, shuffle=True)
        if dev_filename is not None: 
            self.dev_dataset = create_dataset(
                sentences=self.dev_file[4].to_numpy(),
                aspect_categories = self.dev_file[1].to_numpy(),
                target_terms=self.dev_file[2].to_numpy(),
                labels = self.dev_file[0].to_numpy(),
                tokenizer = self.tokenizer,
                max_len_token = self.max_length)
            self.val_loader = DataLoader(self.dev_dataset, batch_size= self.bs, shuffle=True)
            
        #----------------MODEL & HYPEPARAMETERS------------------#
        # Define the devide and the model
        self.device = device
        self.model = SentimentCheck().to(self.device)
    
        # Hyperparameters
        self.epochs = 15
        self.optimizer = optim.Adam(self.model.parameters(), lr=2e-5)
        self.total_steps = len(self.train_loader) * self.epochs
        self.scheduler = lr_scheduler.LinearLR(self.optimizer, total_iters=self.total_steps) # reduce lr gradually to 0
        self.loss_fn = nn.CrossEntropyLoss().to(self.device)
               
        #----------------TRAIN & VALIDATE MODEL------------------#
        record = defaultdict(list)
        best_val_acc = 0
        for epoch in tqdm(range(self.epochs)):
            print()
            print('=' * 10, f'Epoch {epoch + 1}/{self.epochs}', '=' * 10)
            print()
            
            #----------------TRAINING------------------#
            self.model.train()
            losses = []
            correct_predictions = 0
            
            for batch in self.train_loader:
                # Get inputs for model
                input_ids = batch["input_ids"].to(self.device)
                attention_mask = batch["attention_mask"].to(self.device)
                labels = batch["labels"].to(self.device)
                # Predict and calculate loss
                outputs = self.model(input_ids = input_ids, attention_mask = attention_mask)
                _, preds = torch.max(outputs, dim=1)
                loss = self.loss_fn(outputs, labels)
                correct_predictions += torch.sum(preds == labels)
                losses.append(loss.item())
                # Backprop
                loss.backward()
                # Gradient clipping
                nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                # Take next step
                self.optimizer.step()
                self.scheduler.step()
                
            train_acc, train_loss = correct_predictions.double() / len(self.trainfile), np.mean(losses)        
            print(f'Train loss {train_loss} - Training accuracy {train_acc}')
            print()
            
            #----------------VALIDATION------------------#
            if self.val_loader is not None:
                self.model = self.model.eval()    
                losses = []
                correct_predictions = 0
                
                with torch.no_grad():
                    for batch in self.val_loader:
                        # Get inputs for model
                        input_ids = batch["input_ids"].to(self.device)
                        attention_mask = batch["attention_mask"].to(self.device)
                        labels = batch["labels"].to(self.device)
                        # Predict and calculate loss
                        outputs = self.model(input_ids = input_ids, attention_mask = attention_mask)
                        _, preds = torch.max(outputs, dim=1)
                        loss = self.loss_fn(outputs, labels)
                        correct_predictions += torch.sum(preds == labels)
                        losses.append(loss.item())
                        
                val_acc, val_loss = correct_predictions.double() / len(self.dev_file), np.mean(losses)
                print(f'Val loss {val_loss} - Val accuracy {val_acc}')
                print()
                
            #----------------RECORD ACCURACY & SAVE BEST MODEL------------------#
            record['train_acc'].append(train_acc)
            record['train_loss'].append(train_loss)
                
            if self.val_loader is not None:
                record['val_acc'].append(val_acc)
                record['val_loss'].append(val_loss)
                
            if (self.val_loader is not None) and (val_acc > best_val_acc):
                torch.save(self.model.state_dict(), 'best_model.pth')
                best_val_acc = val_acc
        
    
    ###############################################################################
    def predict(self, data_filename: str, device: torch.device):
        """Predicts class labels for the input instances in file 'datafile'
        Returns the list of predicted labels
        PLEASE:
          - DO NOT CHANGE THE SIGNATURE OF THIS METHOD
          - PUT THE MODEL and DATA on the specified device! Do not use another device
        """
        
        #----------------LOAD FILE & PREPROCESS----------------#
        data_file = pd.read_csv(data_filename, sep='\t', header=None)
        self.data_file = preprocessing(data_file)
        self.dataset = create_dataset(
            sentences=self.data_file[4].to_numpy(),
            aspect_categories = self.data_file[1].to_numpy(),
            target_terms=self.data_file[2].to_numpy(),
            labels = self.data_file[0].to_numpy(),
            tokenizer = self.tokenizer,
            max_len_token = self.max_length)
        self.data_loader = DataLoader(self.dataset, batch_size= self.bs, shuffle=False)
        
        #----------------LOAD MODEL------------------#
        self.device = device
        self.model = SentimentCheck().to(self.device)
        self.model.load_state_dict(torch.load('best_model.pth'))

        #----------------PREDICTING------------------#
        self.model.eval()
        output_labels = []
        label_decode = {0:"negative",1:"neutral",2:"positive"}
        for batch in self.data_loader:
            input_ids = batch["input_ids"].to(self.device)
            attention_mask = batch["attention_mask"].to(self.device)
            labels = batch["labels"].to(self.device)
            with torch.no_grad():
                outputs = self.model(input_ids=input_ids,attention_mask=attention_mask)
            logits = F.softmax(outputs, dim=1)
            logits = logits.detach().cpu().numpy()
            outputs = np.argmax(logits, axis=1) # label in number 0,1,2
            for label in outputs:
                output_labels.append(label_decode[label]) # label in text "negative","neutral","positive"
        return np.array(output_labels)

In [ ]:
# Execute model

def set_reproducible():
    # The below is necessary to have reproducible behavior.
    import random as rn
    import os
    os.environ['PYTHONHASHSEED'] = '0'
    # The below is necessary for starting Numpy generated random numbers
    # in a well-defined initial state.
    np.random.seed(17)
    # The below is necessary for starting core Python generated random numbers
    # in a well-defined state.
    rn.seed(12345)



def load_label_output(filename):
    with open(filename, 'r', encoding='UTF-8') as f:
        return [line.strip().split("\t")[0] for line in f if line.strip()]



def eval_list(glabels, slabels):
    if (len(glabels) != len(slabels)):
        print("\nWARNING: label count in system output (%d) is different from gold label count (%d)\n" % (
        len(slabels), len(glabels)))
    n = min(len(slabels), len(glabels))
    incorrect_count = 0
    for i in range(n):
        if slabels[i] != glabels[i]: incorrect_count += 1
    acc = (n - incorrect_count) / n
    return acc*100



def train_and_eval(classifier, trainfile, devfile, testfile, run_id, device):
    print(f"\nRUN: {run_id}")
    print("  %s.1. Training the classifier..." % str(run_id))
    classifier.train(trainfile, devfile, device)
    print()
    print("  %s.2. Eval on the dev set..." % str(run_id), end="")
    slabels = classifier.predict(devfile, device)
    glabels = load_label_output(devfile)
    devacc = eval_list(glabels, slabels)
    print(" Acc.: %.2f" % devacc)
    testacc = -1
    if testfile is not None:
        # Evaluation on the test data
        print("  %s.3. Eval on the test set..." % str(run_id), end="")
        slabels = classifier.predict(testfile)
        glabels = load_label_output(testfile)
        testacc = eval_list(glabels, slabels)
        print(" Acc.: %.2f" % testacc)
    print()
    return (devacc, testacc)


if __name__ == "__main__":
    device_name = 'cuda'
    device = torch.device(device_name)
    n_runs = 5 #args.n_runs
    set_reproducible()
    datadir = "../data/"
    trainfile =  datadir + "traindata.csv"
    print(trainfile)
    devfile =  datadir + "devdata.csv"
    testfile = None
    # testfile = datadir + "testdata.csv"

    # Runs
    start_time = time.perf_counter()
    devaccs = []
    testaccs = []
    for i in range(1, n_runs+1):
        classifier =  Classifier()
        devacc, testacc = train_and_eval(classifier, trainfile, devfile, testfile, i, device)
        devaccs.append(np.round(devacc,2))
        testaccs.append(np.round(testacc,2))
    print('\nCompleted %d runs.' % n_runs)
    total_exec_time = (time.perf_counter() - start_time)
    print("Dev accs:", devaccs)
    print("Test accs:", testaccs)
    print()
    print("Mean Dev Acc.: %.2f (%.2f)" % (np.mean(devaccs), np.std(devaccs)))
    print("Mean Test Acc.: %.2f (%.2f)" % (np.mean(testaccs), np.std(testaccs)))
    print("\nExec time: %.2f s. ( %d per run )" % (total_exec_time, total_exec_time / n_runs))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


../data/traindata.csv

RUN: 1
  1.1. Training the classifier...


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 0/15 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2302: FutureWarning: The `pad_to_max_l


========== Epoch 1/15 ==========

Train loss 0.7451464720862977 - Training accuracy 0.7059214903526281

Val loss 0.5394221792618433 - Val accuracy 0.8138297872340425



  7%|▋         | 1/15 [00:12<02:52, 12.30s/it]

...

Val loss 0.8733855766137518 - Val accuracy 0.8457446808510638


========== Epoch 15/15 ==========

Train loss 0.11742894566541122 - Training accuracy 0.9753825681969395



100%|██████████| 15/15 [02:52<00:00, 11.47s/it]

Val loss 1.058729358536463 - Val accuracy 0.8351063829787234


  5.2. Eval on the dev set...


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2302: FutureWarning: The `pad_to_max_length` argument is deprecated and wil

 Acc.: 88.83


Completed 5 runs.
Dev accs: [86.97, 86.44, 86.7, 87.23, 88.83]
Test accs: [-1, -1, -1, -1, -1]

Mean Dev Acc.: 87.23 (0.84)
Mean Test Acc.: -1.00 (0.00)

Exec time: 892.20 s. ( 178 per run )
